# Data

- https://vk.com/u_samovaraa?w=wall-81871567_61842
- https://vk.com/vkys_nos?w=wall-41960737_13333
- https://vk.com/receptik_kulinar?w=wall-59496329_52708
- https://vk.com/lisimnik_cake?w=wall-82240292_25648
- https://vk.com/kingcook?w=wall-59442940_11047
- https://vk.com/u_samovaraa?w=wall-81871567_61917
- https://vk.com/quickrecipes?w=wall-61337543_5892
- https://vk.com/namenuru?w=wall-36303114_56579
- https://vk.com/vegan_cookbook?w=wall-43818640_25903
- https://vk.com/multivarka_cook?w=wall-51300483_11948

In [12]:
from glob import iglob as list_paths


def load_text(path):
    with open(path) as file:
        return file.read()


texts = [
    load_text(_)
    for _ in list_paths('texts/*.txt')
]

In [13]:
def make_translation(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation(
    '‑–—−',
    '----'
)


def preprocess(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\xad', '')
    text = text.translate(DASHES_TRANSLATION)
    return text


texts = [preprocess(_) for _ in texts]

In [14]:
print(texts[0])

Салат с ананасами и курицей 🍍

Ингредиенты:

● Филе куриной грудки 1 шт.
● Шампиньоны 300 г
● Кукуруза консервированная 1 банка
● Сыр 200 г
● Лук репчатый 1 головка
● Чеснок 2 зубчика
● Ананас консервированный 1 банка
● Майонез по вкусу

Приготовление:

1. Обжарить грибы с луком с добавить немного соли. Старайтесь не резать грибы слишком мелко.
2. Отварить куриную грудку без кожи, нарезать кусочками и добавить к ней консервированную кукурузу.
3. Сыр также нарезать маленькими кубиками или натереть на крупной терке. Добавить к курице с кукурузой.
4. Смешать все ингредиенты с консервированным ананасом и заправить салат майонезом с чесноком.

Приятного аппетита!


In [15]:
import re


def find_ingredient_sections(text):
    return re.findall(r'ингредиенты:(.+?)приготовление', text, re.I | re.S)



def maybe_ingredient(line):
    match = re.search(r'\d', line)
    size = len(line) <= 50
    return match and size


lines = []
for text in texts:
    sections = find_ingredient_sections(text)
    for section in sections:
        for line in section.splitlines():
            if maybe_ingredient(line):
                lines.append(line)

In [16]:
from random import seed, sample

seed(1)
sample(lines, 20)

['-1 луковица ',
 '1 стакан муки',
 '2 яйца',
 'Соль - 2 ст.ложки',
 '-100 грамм сыра ',
 'вишни морож.3 горсти,',
 'Перец болгарский - 3 Штуки',
 'сок 1/2 лимона ',
 '1 репчатый лук ',
 'соль - 1 Чайная ложка (для теста) ',
 '● Растительное масло 1 ст.л.',
 'Сыр твердый - 4 ст. л.',
 '100 г сливочного масла,',
 '1 зубчик чеснока',
 '● салями 100 г ',
 '-Огурец - 1 шт.',
 'растительное масло - 3 ст. ложки ',
 '4 средних спелых банана',
 'Кинза 2 г ',
 'Репчатый лук - 2 Штуки (среднего размера)']

In [17]:
len(lines)

4054

# Grammar

In [61]:
from yargy.tokenizer import MorphTokenizer

TOKENIZER = MorphTokenizer()
list(TOKENIZER('рулетики'))

[MorphToken('рулетики',
            [0, 8),
            'RU',
            [Form('рулетик', Grams(NOUN,inan,masc,nomn,plur)),
             Form('рулетик', Grams(NOUN,accs,inan,masc,plur)),
             Form('рулетик', Grams(NOUN,inan,masc,nomn,plur)),
             Form('рулетик', Grams(NOUN,accs,inan,masc,plur)),
             Form('рулётик', Grams(NOUN,inan,masc,nomn,plur)),
             Form('рулётик', Grams(NOUN,accs,inan,masc,plur))])]

In [63]:
# %run -n main.py
# parser = Parser(INGREDIENT)
# seed(1)
# for line in sample(lines, 100):
#     matches = list(parser.findall(line))
#     spans = [_.span for _ in matches]
#     show_markup(line, spans)
# #     if matches:
# #         match = matches[0]
# #         display(match.tree.as_dot)
# #         display(match.fact)